# Data Preparation 

In [1]:
#Import packages
import pandas as pd
import csv
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import numpy as np
import folium
#from PIL import Image
from sklearn.preprocessing import MinMaxScaler
from scipy import stats
from scipy.stats import anderson
from scipy.stats import norm
from matplotlib import pylab

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
warnings.filterwarnings('ignore')
%matplotlib inline

In [2]:
#import dataset created in Data Understanding
accidents = pd.read_csv(r"C:\Users\DETCAO03\V-Case study\02_Dataset\Used\Accidents.csv",low_memory=False, encoding='utf-8')

## Select features clean and construct data

In [3]:
accidents.drop(['Location_Easting_OSGR', 'Location_Northing_OSGR', 'Accident_Index', 'LSOA_of_Accident_Location', 'Police_Force', 'Local_Authority_(District)',
                                   'Local_Authority_(Highway)', 'Junction_Detail', '2nd_Road_Class', '2nd_Road_Number', 
                                   'Did_Police_Officer_Attend_Scene_of_Accident', '1st_Road_Number',
                'Pedestrian_Crossing-Physical_Facilities'], axis=1, inplace=True)

In [4]:
#date
accidents['Date_time'] =  accidents['Date'] +' '+ accidents['Time']
accidents['Date_time'] = pd.to_datetime(accidents["Date_time"])
accidents.drop(['Date','Time'],axis =1 , inplace=True)

#coordinates
accidents["LatLon"] = list(zip(accidents["Latitude"], accidents["Longitude"]))
accidents.drop(['Latitude','Longitude'],axis=1 , inplace=True)

#fill missing region values with "mode" and binary encoding of region
accidents["Region"] = accidents["Region"].fillna(accidents["Region"].mode()[0])
accidents = pd.get_dummies(accidents, columns=['Region'], drop_first=True)

#drop duplicates (151)
accidents.dropna(inplace=True)

#Normalisation
scaler = MinMaxScaler()
accidents[["Number_of_Vehicles","Number_of_Casualties","Day_of_Week","1st_Road_Class","Road_Type","Speed_limit","Junction_Control","Pedestrian_Crossing-Human_Control","Light_Conditions","Weather_Conditions","Road_Surface_Conditions","Special_Conditions_at_Site","Carriageway_Hazards","Urban_or_Rural_Area"]] = scaler.fit_transform(accidents[["Number_of_Vehicles","Number_of_Casualties","Day_of_Week","1st_Road_Class","Road_Type","Speed_limit","Junction_Control","Pedestrian_Crossing-Human_Control","Light_Conditions","Weather_Conditions","Road_Surface_Conditions","Special_Conditions_at_Site","Carriageway_Hazards","Urban_or_Rural_Area"]])


In [5]:
#Road

#accidents["1st_Road_Class"] = accidents["1st_Road_Class"].map(lambda x: {1:"Motorway",2:"A(M)",3:"A",4:"B",5:"C", 6:"Unclassified"}.get(x,x))
#accidents["1st_Road_Number"] = accidents["1st_Road_Number"].astype(str)
#accidents["Roadnumber"] = accidents.apply(lambda x: "Unclassified" if x["1st_Road_Class"]=="Unclassified" else ("A"+x["1st_Road_Number"]+"(M)" if x["1st_Road_Class"]=="A(M)" else x["1st_Road_Class"][0] + x["1st_Road_Number"]), axis = 1)
#accidents.drop(['1st_Road_Class','1st_Road_Number'],axis=1 , inplace=True)

In [9]:
accidents.head()

,Accident_Severity,Number_of_Vehicles,Number_of_Casualties,Day_of_Week,1st_Road_Class,Road_Type,Speed_limit,Junction_Control,Pedestrian_Crossing-Human_Control,Light_Conditions,Weather_Conditions,Road_Surface_Conditions,Special_Conditions_at_Site,Carriageway_Hazards,Urban_or_Rural_Area,Date_time,LatLon,Region_East of England,Region_London,Region_North East (England),Region_North West (England),Region_Scotland,Region_South East (England),Region_South West (England),Region_Wales,Region_West Midlands (England),Region_Yorkshire and The Humber
0,2,0.000000,0.0,0.333333,0.4,0.625,0.428571,0.0,0.333333,0.0,0.3,0.500000,0.125,0.125,0.0,2005-04-01 17:42:00,"(51.489096, -0.19117)",0,1,0,0,0,0,0,0,0,0
1,3,0.000000,0.0,0.500000,0.6,0.250,0.428571,0.6,0.333333,0.5,0.2,0.333333,0.125,0.125,0.0,2005-05-01 17:36:00,"(51.520075, -0.211708)",0,1,0,0,0,0,0,0,0,0
2,3,0.015152,0.0,0.666667,0.8,0.625,0.428571,0.0,0.333333,0.5,0.2,0.333333,0.125,0.125,0.0,2005-06-01 00:15:00,"(51.525301, -0.206458)",0,1,0,0,0,0,0,0,0,0
3,3,0.000000,0.0,0.833333,0.4,0.625,0.428571,0.0,0.333333,0.0,0.2,0.333333,0.125,0.125,0.0,2005-07-01 10:35:00,"(51.482442, -0.173862)",0,1,0,0,0,0,0,0,0,0
4,3,0.000000,0.0,0.166667,1.0,0.625,0.428571,0.0,0.333333,1.0,0.2,0.500000,0.125,0.125,0.0,2005-10-01 21:13:00,"(51.495752, -0.156618)",0,1,0,0,0,0,0,0,0,0


In [6]:
accidents.shape

(1780502, 27)

In [7]:
#final dataset for Modeling
accidents.to_csv(r"C:\Users\DETCAO03\V-Case study\02_Dataset\Used\Cleaned_dataset_accidents.csv", index=False, encoding='utf-8')

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.metrics import log_loss